In [83]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    #print(questions[:5])
    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('answers_squad_format.json')
#val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [84]:
train_answers[:2]

[{'text': 'Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer�s machines are up and running much faster, saving them from costly downtime.\n \nFor a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.',
  'answer_start': 0},
 {'text': 'Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects c

In [85]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
#add_end_idx(val_answers, val_contexts)

In [86]:
train_answers[:5]

[{'text': 'Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer�s machines are up and running much faster, saving them from costly downtime.\n \nFor a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.',
  'answer_start': 0,
  'answer_end': 779},
 {'text': 'Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which 

In [87]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
#val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [88]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [89]:
len(train_encodings['input_ids'])

45

In [90]:
train_encodings['input_ids'][0]

[101,
 2108,
 2583,
 2000,
 6509,
 6304,
 19512,
 1010,
 2326,
 20202,
 2071,
 2490,
 2037,
 6304,
 2012,
 2151,
 2051,
 1998,
 2013,
 5973,
 1010,
 2302,
 9352,
 2383,
 2000,
 3942,
 1996,
 8013,
 2609,
 1012,
 6556,
 3229,
 2071,
 2036,
 2393,
 2000,
 2488,
 7374,
 2326,
 20202,
 2007,
 1996,
 4072,
 2592,
 2005,
 2037,
 8518,
 1012,
 4209,
 2055,
 1996,
 3698,
 2030,
 2291,
 7561,
 1999,
 2062,
 6987,
 25828,
 13169,
 7070,
 2051,
 2138,
 3223,
 8622,
 3033,
 2030,
 2060,
 3941,
 2064,
 3641,
 1010,
 4810,
 1998,
 2716,
 2247,
 1012,
 9308,
 1010,
 2349,
 2000,
 1037,
 7820,
 4668,
 2051,
 1010,
 1996,
 6304,
 6681,
 2024,
 2039,
 1998,
 2770,
 2172,
 5514,
 1010,
 7494,
 2068,
 2013,
 17047,
 2091,
 7292,
 1012,
 2005,
 1037,
 3698,
 1998,
 3941,
 7751,
 1010,
 2478,
 6556,
 3229,
 2036,
 2965,
 2008,
 1996,
 2168,
 2193,
 1997,
 2326,
 20202,
 2071,
 2490,
 2062,
 6304,
 2030,
 3749,
 3176,
 2578,
 1012,
 1037,
 5851,
 6556,
 4434,
 2000,
 5500,
 10394,
 1998,
 3941,
 2003,
 2036,

In [91]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. remote access could also help to better prepare service technicians with the necessary information for their tasks. knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. moreover, due to a shorter reaction time, the customers machines are up and running much faster, saving them from costly downtime. for a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services. a secure remote connection to distributed machinery and equipment is also the basis for many new concepts and services within industry 4. 0 such as predictive maintenance, where a secure connection is established to 

In [92]:
train_encodings.char_to_token(0, train_answers[0]['answer_start'])

1

In [99]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        go_back=1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - go_back)
            go_back+=1
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
#add_token_positions(val_encodings, val_answers)

In [100]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [95]:
train_encodings['start_positions'][:10]

[1, 1, 1, 148, 512, 1, 512, 323, 1, 512]

In [5]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
#val_dataset = SquadDataset(val_encodings)

In [114]:
print(type(train_dataset))

<class '__main__.SquadDataset'>


In [101]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to

In [102]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

c:\Users\NIKITA_PC\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [103]:


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    loop=tqdm(train_loader)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

model.eval()

Epoch 2: 100%|██████████| 3/3 [11:07<00:00, 222.51s/it, loss=<built-in method item of Tensor object at 0x000000913501B650>]


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [104]:
model_path='model/distilbert-custom'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('model/distilbert-custom\\tokenizer_config.json',
 'model/distilbert-custom\\special_tokens_map.json',
 'model/distilbert-custom\\vocab.txt',
 'model/distilbert-custom\\added_tokens.json',
 'model/distilbert-custom\\tokenizer.json')

In [144]:
# def get_answer(tokenizer, question, context):
#   inputs = tokenizer([question], [context])
#   outputs = model(inputs_ids,attention_mask=attention mask)
#   start_position = torch.argmax(outputs['start_logits'], dim=1)
#   end_position = torch.argmax(outputs['end_logits'], dim=1)
#   answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
#   return tokenizer.decode(answer).strip()
def get_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
    with torch.no_grad():
        output_start, output_end = model(**inputs)
        
        answer_start = torch.argmax(output_start)  
        answer_end = torch.argmax(output_end) 

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

        return(answer)

In [145]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is Foundation of Smart Manufacturing?"
print(f"Predicted Answer: {get_answer(question, context)}")
#input_ids=test_encodings['input_ids'].to(device)
#outputs=model(test_encodings['input_ids'],test_encodings['attention_mask'])

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [141]:
# 
def get_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
    with torch.no_grad():
        output_start, output_end = model(**inputs)
        
        answer_start = torch.argmax(output_start)  
        answer_end = torch.argmax(output_end) 

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

        return(answer)
        



  0%|          | 0/29 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (10) must match the size of tensor b (512) at non-singleton dimension 1

In [55]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model1 = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

print("Tokenizer and model successfully initialized.")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to

Tokenizer and model successfully initialized.


In [59]:
import numpy as np
#question, passage,
#def distilbert_question_answer( max_len=500):
passage="""Contact us for Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. under Samarth Udyog Mission of the Ministry of Heavy Industry , Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative .We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi"""
question ="What are details of Internship 2023? "
max_len=500
inputs = tokenizer.encode_plus(question, passage, add_special_tokens=True, max_length=max_len, truncation=True, return_tensors='pt',return_attention_mask=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

start_scores, end_scores = model1(input_ids, attention_mask=attention_mask)
print(start_scores)
print(end_scores)
    # start_scores = start_scores.cpu().numpy().flatten()
    # end_scores = end_scores.cpu().numpy().flatten()

    # answer_start_index = np.argmax(start_scores)
    # answer_end_index = np.argmax(end_scores)

    # tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
    # answer = tokens[answer_start_index]

    # for i in range(answer_start_index + 1, answer_end_index + 1):
    #     if tokens[i][0:2] == '##':
    #         answer += tokens[i][2:]
    #     else:
    #         answer += ' ' + tokens[i]

    # start_token_score = np.round(start_scores[answer_start_index], 2)
    # end_token_score = np.round(end_scores[answer_end_index], 2)

    # if (
    #     answer_start_index == 0
    #     or start_token_score < 0
    #     or answer == '[SEP]'
    #     or answer_end_index < answer_start_index
    # ):
    #     answer = "Sorry, I could not find an answer in the passage."

    # return answer_start_index, answer_end_index, start_token_score, end_token_score, answer
    #return start_scores,end_scores

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_logits
end_logits


In [56]:
import numpy as np
#
def distilbert_question_answer( question, passage,max_len=500):
# passage="""Contact us for Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. under Samarth Udyog Mission of the Ministry of Heavy Industry , Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative .We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi"""
# question ="What are details of Internship 2023? "
# max_len=500
    inputs = tokenizer.encode_plus(question, passage, add_special_tokens=True, max_length=max_len, truncation=True, return_tensors='pt',return_attention_mask=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    start_scores, end_scores = model1(input_ids, attention_mask=attention_mask)
    print(start_scores)
    print(end_scores)
    start_scores = start_scores.cpu().numpy().flatten()
    end_scores = end_scores.cpu().numpy().flatten()

    answer_start_index = np.argmax(start_scores)
    answer_end_index = np.argmax(end_scores)

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
    answer = tokens[answer_start_index]

    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    start_token_score = np.round(start_scores[answer_start_index], 2)
    end_token_score = np.round(end_scores[answer_end_index], 2)

    if (
        answer_start_index == 0
        or start_token_score < 0
        or answer == '[SEP]'
        or answer_end_index < answer_start_index
    ):
        answer = "Sorry, I could not find an answer in the passage."

    return answer_start_index, answer_end_index, start_token_score, end_token_score, answer
    #return start_scores,end_scores

In [57]:
passage="""Contact us for Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. under Samarth Udyog Mission of the Ministry of Heavy Industry , Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative .We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi"""
print (f'Length of the passage: {len(passage.split())} words')

Length of the passage: 511 words


In [58]:
question ="What are details of Internship 2023? "

answer_tuple = distilbert_question_answer(question, passage)
print(answer_tuple)
# answer = answer_tuple[-1]  # Extract the answer string from the tuple

# print("Question:", question)
# print("Answer:", answer)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_logits
end_logits


AttributeError: 'str' object has no attribute 'cpu'